In [4]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from io import StringIO

# Define the range of seasons to process
start_year = 1980  
end_year = 1999    

# Initialize WebDriver (Ensure geckodriver is installed and in your PATH)
wd = webdriver.Firefox()

def scrape_season(year):
    """Scrapes NBA player stats for a given season and saves to CSV, skipping empty seasons."""
    url = f"https://www.espn.com/nba/stats/player/_/season/{year}/seasontype/2"
    filename = f'nba_player_stats_{year-1}-{year}.csv'
    
    print(f"Processing season {year-1}-{year}...")

    # Open the URL
    wd.get(url)
    time.sleep(3)  # Allow page to load

    # Check if "No Data Available" message is present
    try:
        no_data_message = wd.find_element(By.XPATH, "//div[contains(text(), 'No Data Available')]")
        if no_data_message:
            print(f"Skipping {year-1}-{year}: No Data Available")
            return  # Skip this season
    except NoSuchElementException:
        pass  # No error means data is available, continue scraping

    # Function to click "Show More" until all data is loaded
    def click_show_more():
        while True:
            try:
                # Locate the "Show More" link inside the div
                show_more_link = wd.find_element(By.XPATH, "//div[contains(@class, 'loadMore')]//a[contains(@class, 'loadMore__link')]")
                
                # Scroll into view
                wd.execute_script("arguments[0].scrollIntoView();", show_more_link)
                time.sleep(1)  # Allow scrolling time
                
                # Click using JavaScript (ensures it works)
                wd.execute_script("arguments[0].click();", show_more_link)
                time.sleep(2)  # Allow content to load
            except NoSuchElementException:
                print("No more 'Show More' button found. Page is fully loaded.")
                break
            except ElementClickInterceptedException:
                print("Click intercepted. Retrying after a short wait...")
                time.sleep(2)

    # Click "Show More" until everything is loaded
    click_show_more()

    # Extract player names
    try:
        player_elements = wd.find_elements(By.XPATH, "//tr[contains(@class, 'Table__TR')]//td[2]//a")
        names = [element.text for element in player_elements if element.text]
        print(f"Number of player names extracted: {len(names)}")
        if len(names) == 0:
            print(f"Skipping {year-1}-{year}: No player data found.")
            return  # Skip this season
    except Exception as e:
        print(f"Skipping {year-1}-{year}: Error extracting player names:", e)
        return  # Skip this season

    # Extract tables using pandas (fixing the warning)
    html_source = wd.page_source
    try:
        tables = pd.read_html(StringIO(html_source))
    except ValueError:
        print(f"Skipping {year-1}-{year}: No tables found.")
        return  # Skip this season

    # Ensure tables exist
    if len(tables) < 2:
        print(f"Skipping {year-1}-{year}: Expected at least 2 tables but found {len(tables)}")
        return  # Skip this season

    # Extract player data
    players = tables[0]
    stats = tables[1]

    # Drop 'RK' column if it exists
    if 'RK' in players.columns:
        players = players.drop(columns=['RK'])

    # Add extracted player names
    players['Name'] = names

    # Merge player and stats DataFrames
    df = pd.concat([players, stats], axis=1)

    # Display the final DataFrame
    print(df.head())

    # Save to CSV
    df.to_csv(filename, index=False)
    print(f"Data saved to {filename}")

# Loop through each season
for year in range(start_year, end_year + 1):
    scrape_season(year)

# Close WebDriver after processing all seasons
wd.quit()
print("All seasons processed successfully.")


Processing season 1979-1980...
Skipping 1979-1980: No Data Available
Processing season 1980-1981...
Skipping 1980-1981: No Data Available
Processing season 1981-1982...
Skipping 1981-1982: No Data Available
Processing season 1982-1983...
Skipping 1982-1983: No Data Available
Processing season 1983-1984...
Skipping 1983-1984: No Data Available
Processing season 1984-1985...
Skipping 1984-1985: No Data Available
Processing season 1985-1986...
Skipping 1985-1986: No Data Available
Processing season 1986-1987...
No more 'Show More' button found. Page is fully loaded.
Number of player names extracted: 7
              Name POS  GP   MIN   PTS  FGM   FGA   FG%  3PM  3PA  ...  FTM  \
0        Fat Lever   G  82  37.2  18.9  7.8  16.7  46.9  0.3  1.1  ...  3.0   
1     Michael Cage   C  80  36.5  15.7  5.7  11.0  52.0  0.0  0.0  ...  4.3   
2  Benoit Benjamin   C  72  31.0  11.5  4.4   9.9  44.9  0.0  0.0  ...  2.6   
3  Blair Rasmussen   C  74  19.2   9.5  3.6   7.7  47.0  0.0  0.0  ...  2.3   